In [87]:
import requests
import pandas as pd
import time, random, json, os
from tqdm import tqdm
from datetime import datetime

In [88]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}

In [89]:
SEARCH_URL = "https://tiki.vn/api/v2/products"
DETAIL_URL = "https://tiki.vn/api/v2/products/{}"
REVIEW_URL = "https://tiki.vn/api/v2/reviews"

In [90]:
os.makedirs("./data_full_2/checkpoint", exist_ok=True)
os.makedirs("./data_full_2/final", exist_ok=True)

In [91]:
def _sleep(a=1.5, b=3.0):
    time.sleep(random.uniform(a, b))

In [92]:
def safe_request(url, params=None, max_retry=5, base_sleep=2):
    """Request có retry/backoff"""
    for i in range(max_retry):
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=45)
            if r.status_code == 200:
                return r
            else:
                print(f"[WARN] {r.status_code} @ {url} | retry {i+1}")
        except (requests.exceptions.ConnectTimeout, requests.exceptions.ReadTimeout):
            print(f"[TIMEOUT] {url} | retry {i+1}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] {e} | retry {i+1}")
        time.sleep(base_sleep * (1.5 ** i) + random.uniform(0, 2))
    return None


In [93]:
def bucket_price(price):
    """Chia price thành bucket rời rạc để dùng cho phân phối theo context (JS divergence, v.v.)"""
    if price is None:
        return None
    try:
        p = float(price)
    except (TypeError, ValueError):
        return None

    # Bạn có thể chỉnh lại ngưỡng cho phù hợp dữ liệu Việt Nam
    if p < 100_000:
        return "low_<100k"
    elif p < 300_000:
        return "mid_100k_300k"
    elif p < 700_000:
        return "mid_high_300k_700k"
    else:
        return "high_>700k"

In [94]:
def extract_time_features(created_at):
    """
    created_at: timestamp kiểu epoch (int hoặc str).
    Trả về: dict {event_time, hour, day_of_week, is_weekend, time_slot, context_key}
    """
    if created_at is None:
        return {
            "event_time": None,
            "hour": None,
            "day_of_week": None,
            "is_weekend": None,
            "time_slot": None,
            "context_key": None,
        }

    try:
        ts = int(created_at)
    except (TypeError, ValueError):
        try:
            dt = datetime.fromisoformat(str(created_at))
        except Exception:
            return {
                "event_time": None,
                "hour": None,
                "day_of_week": None,
                "is_weekend": None,
                "time_slot": None,
                "context_key": None,
            }
    else:
        dt = datetime.fromtimestamp(ts)

    hour = dt.hour
    dow = dt.weekday()  # 0=Mon, 6=Sun
    is_weekend = 1 if dow >= 5 else 0
    #daytime
    if 5 <= hour < 11:
        time_slot = "morning"   # breakfast
    elif 11 <= hour < 14:
        time_slot = "noon"      # lunch
    elif 14 <= hour < 18:
        time_slot = "afternoon"
    elif 18 <= hour < 22:
        time_slot = "evening"   # dinner
    else:
        time_slot = "late_night"

    context_key = f"{time_slot}_wd{dow}"  

    return {
        "event_time": dt.isoformat(), 
        "hour": hour,
        "day_of_week": dow,
        "is_weekend": is_weekend,
        "time_slot": time_slot,
        "context_key": context_key,
    }


In [95]:
def get_product_detail_full(product_id):
    """Lấy chi tiết sản phẩm + bổ sung feature cho CoFARS"""
    url = DETAIL_URL.format(product_id)
    r = safe_request(url)
    if not r:
        return {}
    try:
        data = r.json()
    except json.JSONDecodeError:
        print(f"[BAD JSON] {product_id}")
        return {}

    cats = data.get("categories")
    categories = None
    category_root = None
    category_leaf = None

    # categories thường là list từ root -> leaf
    if isinstance(cats, list) and cats:
        names = [c.get("name", "") for c in cats if isinstance(c, dict)]
        categories = " > ".join(names) if names else None
        category_root = names[0] if names else None
        category_leaf = names[-1] if names else None
    elif isinstance(cats, dict):
        categories = cats.get("name")
        category_leaf = categories

    cur_seller = data.get("current_seller")
    seller = cur_seller.get("name") if isinstance(cur_seller, dict) else None
    brand = (data.get("brand") or {}).get("name") or data.get("brand_name")

    return {
        "brand_detail": brand,
        "seller_detail": seller,
        "categories": categories,
        "category_root": category_root,
        "category_leaf": category_leaf,
        "short_description": data.get("short_description"),
    }


In [96]:
def get_reviews(product_id, max_pages=2):
    """Lấy tối đa 2 trang review / sản phẩm, có thêm context time features cho CoFARS"""
    reviews = []
    for page in range(1, max_pages + 1):
        params = {"product_id": product_id, "page": page, "limit": 20}
        r = safe_request(REVIEW_URL, params=params)
        if not r:
            break

        try:
            data = r.json()
        except json.JSONDecodeError:
            print(f"[BAD JSON] {product_id} page {page}")
            break

        rv_list = data.get("data", [])
        if not rv_list:
            break

        for rv in rv_list:
            # created_by đôi khi = null
            user = rv.get("created_by")
            if not isinstance(user, dict):
                user = {}

            created_at = rv.get("created_at")
            time_feats = extract_time_features(created_at)

            reviews.append({
                "product_id": product_id,
                "customer_id": user.get("id"),
                "customer_name": user.get("name"),
                "rating": rv.get("rating"),
                "title": rv.get("title"),
                "content": rv.get("content"),
                "thank_count": rv.get("thank_count"),
                "created_at_raw": created_at,
                # ==== CoFARS-style time context features ====
                "event_time": time_feats["event_time"],
                "hour": time_feats["hour"],
                "day_of_week": time_feats["day_of_week"],
                "is_weekend": time_feats["is_weekend"],
                "time_slot": time_feats["time_slot"],
                "context_key": time_feats["context_key"],
            })

        _sleep(0.5, 1.2)

    return reviews


In [97]:
def search_tiki_full(keyword: str, group: str, max_pages: int = 10, checkpoint_every=1000):
    """Crawl Products + Details + 2-page Reviews với checkpoint & feature cho CoFARS"""
    products_out, reviews_out = [], []
    checkpoint_id = 0

    for page in tqdm(range(1, max_pages + 1), desc=f"{group} → {keyword[:25]}"):
        params = {"q": keyword, "page": page}
        r = safe_request(SEARCH_URL, params=params)
        if not r:
            break
        data = r.json().get("data", [])
        if not data:
            break

        for p in data:
            pid = p.get("id")
            price = p.get("price")
            price_bucket = bucket_price(price)

            base_info = {
                "id": pid,
                "name": p.get("name"),
                "price": price,
                "price_bucket": price_bucket,      
                "rating": p.get("rating_average"),
                "review_count": p.get("review_count"),
                "brand": (p.get("brand") or {}).get("name") or p.get("brand_name"),
                "seller": (p.get("seller") or {}).get("name"),
                "url": f"https://tiki.vn/{p.get('url_path')}" if p.get("url_path") else None,
                "kw": keyword,
                "group": group,
                "page": page,
            }

            # --- Lấy chi tiết ---
            detail_info = get_product_detail_full(pid)
            base_info.update(detail_info)
            products_out.append(base_info)

            # --- Lấy review ---
            rv = get_reviews(pid, max_pages=2)
            for r_item in rv:
                r_item["kw"] = keyword
                r_item["group"] = group
                reviews_out.append(r_item)

            _sleep(1.5, 3.0)

            # --- Checkpoint ---
            if len(products_out) >= checkpoint_every:
                checkpoint_id += 1
                pd.DataFrame(products_out).to_csv(
                    f"./data_full/checkpoint/products_ckpt_{group[:10]}_{keyword[:10]}_{checkpoint_id}.csv",
                    index=False, encoding="utf-8-sig"
                )
                pd.DataFrame(reviews_out).to_csv(
                    f"./data_full/checkpoint/reviews_ckpt_{group[:10]}_{keyword[:10]}_{checkpoint_id}.csv",
                    index=False, encoding="utf-8-sig"
                )
                print(f"[CHECKPOINT] Saved batch {checkpoint_id} ({len(products_out)} products)")
                products_out.clear()
                reviews_out.clear()

        _sleep(2, 4)

    # Lưu phần còn lại
    if products_out:
        checkpoint_id += 1
        pd.DataFrame(products_out).to_csv(
            f"./data_full/checkpoint/products_ckpt_{group[:10]}_{keyword[:10]}_{checkpoint_id}.csv",
            index=False, encoding="utf-8-sig"
        )
    if reviews_out:
        pd.DataFrame(reviews_out).to_csv(
            f"./data_full/checkpoint/reviews_ckpt_{group[:10]}_{keyword[:10]}_{checkpoint_id}.csv",
            index=False, encoding="utf-8-sig"
        )
    print(f"Hoàn thành keyword: {keyword} ({checkpoint_id} checkpoint)\n")

In [98]:
HOME_LIFE_KEYWORDS = {
    "Dụng cụ nhà bếp": [
        "ấm nước các loại",
        "dụng cụ chứa đựng thực phẩm",
        "dao, kéo và phụ kiện",
        "dụng cụ làm bánh",
        "kệ nhà bếp",
        "bếp nướng, vỉ nướng",
        "phụ kiện nhà bếp",
        "nồi và chảo",
        "bộ hộp cơm và phụ kiện",
        "thùng đựng gạo",
        "nắp đậy và phụ kiện dùng cho lò vi sóng",
        "giấy thấm dầu ăn"
    ],

    "Đồ dùng phòng ăn": [
        "bình lọc, lõi lọc, thiết bị lọc nước",
        "bình, ly uống trà và phụ kiện",
        "bình, ly uống cà phê và phụ kiện",
        "ly, cốc và phụ kiện ly",
        "đũa, muỗng, nĩa",
        "dụng cụ pha chế",
        "phụ kiện phòng ăn",
        "tô, chén, đĩa",
        "bình đựng nước",
        "bình, ly giữ nhiệt"
    ],

    "Nội thất": [
        "nội thất phòng khách",
        "nội thất phòng ngủ",
        "nội thất phòng ăn",
        "nội thất phòng học, làm việc",
        "nội thất trẻ em",
        "nội thất khác"
    ],

    "Đồ dùng phòng ngủ": [
        "bộ ga, ra, drap",
        "chăn, mền và phụ kiện",
        "gối, ruột gối, vỏ gối",
        "nệm và phụ kiện",
        "phụ kiện phòng ngủ"
    ],

    "Trang trí nhà cửa": [
        "decal và giấy dán tường",
        "decal và giấy dán kính",
        "đồng hồ treo tường",
        "tranh trang trí",
        "trang trí nhà cửa khác",
        "vật phẩm phong thủy"
    ],

    "Đèn & thiết bị chiếu sáng": [
        "đèn pin",
        "đèn ngủ",
        "đèn ngoài trời",
        "đèn trần",
        "đèn cảm ứng",
        "đèn khác",
        "đèn diệt côn trùng",
        "đèn xông tinh dầu, tinh dầu và phụ kiện",
        "đèn học và đọc sách"
    ],

    "Ngoài trời & sân vườn": [
        "áo mưa, ô dù và phụ kiện đi mưa",
        "dụng cụ làm vườn",
        "máy xịt rửa và phụ kiện",
        "ngoại thất sân vườn",
        "bể bơi và phụ kiện",
        "dụng cụ ngoài trời khác"
    ],

    "Đồ dùng & thiết bị nhà tắm": [
        "khăn tắm",
        "phụ kiện nhà tắm",
        "kệ nhà tắm",
        "thiết bị phòng tắm",
        "phụ kiện giặt ủi",
        "dụng cụ vệ sinh"
    ],

    "Sửa chữa nhà cửa": [
        "bảo hộ lao động",
        "xe đẩy hàng",
        "sơn, dầu mỡ, hóa chất",
        "đá cắt, que hàn, cưa và lưỡi cưa",
        "bộ dụng cụ đa năng",
        "thang nhôm",
        "dụng cụ sửa chữa nhà cửa",
        "thiết bị đo lường",
        "thùng đồ nghề, túi công cụ"
    ],

    "Nhạc cụ": [
        "đàn piano",
        "amply",
        "đàn organ, keyboard",
        "đàn guitar, ukulele",
        "phụ kiện đàn",
        "nhạc cụ dân tộc",
        "nhạc cụ hơi",
        "phụ kiện nhạc cụ",
        "các loại đàn khác",
        "amplifier",
        "bộ trống drum",
        "đàn kalimba"
    ],

    "Đồ thờ cúng": [
        "bộ đồ thờ",
        "lư trầm, bát hương, bát nhang",
        "bàn thờ",
        "hương, nhang",
        "phụ kiện thờ cúng",
        "đèn thờ",
        "kỷ chén thờ",
        "mâm đồng, mâm ngũ quả",
        "ống cắm hương, nhang",
        "vật dụng trang trí thờ"
    ],

    "Dụng cụ & Thiết bị tiện ích": [
        "bảo vệ nhà cửa",
        "dụng cụ diệt chuột, côn trùng",
        "giá, móc, cây treo",
        "pin và dụng cụ sạc pin",
        "thiết bị điều khiển thông minh",
        "miếng dán đa năng"
    ]
}


In [99]:
print("\nTest \n")
TEST_KWS = ["nồi và chảo", "đèn ngủ", "kệ nhà tắm"]


Test 



In [100]:
df_test_products, df_test_reviews = [], []
for kw in TEST_KWS:
    df_p, df_r = [], []
    print(f"→ Test keyword: {kw}")
    for page in range(1, 3):  # chỉ test 2 trang
        params = {"q": kw, "page": page}
        r = safe_request(SEARCH_URL, params=params)
        if not r:
            continue
        data = r.json().get("data", [])
        for p in data[:3]:  # chỉ lấy 3 sản phẩm / trang
            pid = p.get("id")
            base_info = {
                "id": pid,
                "name": p.get("name"),
                "price": p.get("price"),
                "price_bucket": bucket_price(p.get("price")),
                "rating": p.get("rating_average"),
                "review_count": p.get("review_count"),
                "url": f"https://tiki.vn/{p.get('url_path')}" if p.get("url_path") else None
            }
            detail = get_product_detail_full(pid)
            base_info.update(detail)
            df_p.append(base_info)

            rv = get_reviews(pid, max_pages=1)
            for r_item in rv:
                r_item["kw"] = kw
                df_r.append(r_item)
            _sleep(1, 2)

    if df_p:
        df_test_products.append(pd.DataFrame(df_p))
    if df_r:
        df_test_reviews.append(pd.DataFrame(df_r))

if df_test_products:
    pd.concat(df_test_products, ignore_index=True).to_csv(
        "./data_full_2/test_home_life_products.csv", index=False, encoding="utf-8-sig"
    )
if df_test_reviews:
    pd.concat(df_test_reviews, ignore_index=True).to_csv(
        "./data_full_2/test_home_life_reviews.csv", index=False, encoding="utf-8-sig"
    )

print("Test crawl xong. Kiểm tra file:")
print("→ ./data_full_2/test_home_life_products.csv")
print("→ ./data_full_2/test_home_life_reviews.csv")

→ Test keyword: nồi và chảo
→ Test keyword: đèn ngủ
→ Test keyword: kệ nhà tắm
Test crawl xong. Kiểm tra file:
→ ./data_full_2/test_home_life_products.csv
→ ./data_full_2/test_home_life_reviews.csv


In [101]:
test_product=pd.read_csv("./data_full_2/test_home_life_products.csv")

In [102]:
test_product

,id,name,price,price_bucket,rating,review_count,url,brand_detail,seller_detail,categories,category_root,category_leaf,short_description
0,274054379,Bộ nồi chảo chống dính Elmich baby EL-1168 siz...,281080,mid_100k_300k,4.5,9,https://tiki.vn/bo-noi-chao-chong-dinh-elmich-...,Elmich,Tiki Trading,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,Bộ nồi chảo chống dính sứ Elmich Baby EL-1168M...
1,272359482,Bộ nồi chảo Inox nguyên khối an toàn Baby EL 1...,393610,mid_high_300k_700k,4.3,4,https://tiki.vn/bo-noi-chao-inox-nguyen-khoi-a...,Elmich,Tiki Trading,Bộ nồi chảo,NaN,Bộ nồi chảo,Phù hợp với mọi loại bếpKhả năng bắt từ đỉnh c...
2,132996433,Bộ nồi chảo SL1 4 món quánh 16cm + nồi 20cm + ...,1149000,high_>700k,5.0,1,https://tiki.vn/bo-noi-chao-sl1-4-mon-quanh-16...,Seoulcook,Ant House,Bộ nồi,NaN,Bộ nồi,Bộ sản phẩm bao gồm: - 1 Quánh 1 tay cầm đáy t...
3,274593357,Tặng cặp nhấc nồi silicon cao cấp- Bộ nồi chảo...,1340000,high_>700k,0.0,0,https://tiki.vn/tang-cap-nhac-noi-silicon-cao-...,Kimscook,Ant House,Bộ nồi,NaN,Bộ nồi,Bộ sản phẩm bao gồm:1 Nồi Easy Seoulcook 18 cm...
4,274598354,{Tặng cặp nhấc nồi silicon cao cấp}-Bộ nồi chả...,620000,mid_high_300k_700k,0.0,0,https://tiki.vn/tang-cap-nhac-noi-silicon-cao-...,Seoulcook,Ant House,Bộ nồi,NaN,Bộ nồi,Bộ sản phẩm bao gồm: - 1 Quánh 1 tay cầm đáy t...
5,274598245,Tặng cặp nhấc nồi silicon cap cấp- Bộ nồi chảo...,1200000,high_>700k,0.0,0,https://tiki.vn/tang-cap-nhac-noi-silicon-cap-...,Seoulcook,Ant House,Bộ nồi,NaN,Bộ nồi,Bộ sản phẩm bao gồm: - 1 Quánh 1 tay cầm đáy t...
6,276364671,Đèn ngủ để bàn nhiều hình khối dễ thương - Chí...,56088,low_<100k,5.0,1,https://tiki.vn/den-ngu-de-ban-nhieu-hinh-khoi...,PaKaSa,Phụ Kiện Store 76,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,"Đèn ngủ nhiều hình. Đèn nhỏ gọn, thiết kế cực ..."
7,17885393,Đèn ngủ gỗ - đèn ngủ để bàn - đèn ngủ đầu giườ...,405000,mid_high_300k_700k,5.0,5,https://tiki.vn/den-ngu-go-den-ngu-de-ban-den-...,Goldseee,Monsky Lighting,Đèn ngủ để bàn,NaN,Đèn ngủ để bàn,Đèn ngủ để bàn - đèn ngủ - đèn trang trí phòng...
8,275242036,"Đèn ngủ 2 màu Rhino L802, nhẹ nhàng tiết kiệm ...",44000,low_<100k,5.0,1,https://tiki.vn/den-ngu-2-mau-rhino-l802-nhe-n...,Rhino Store,Rhino Official Store,Đèn ngủ để bàn,NaN,Đèn ngủ để bàn,"Đèn ngủ 2 màu Rhino L802, nhẹ nhàng tiết kiệm ..."
9,276617019,SET 2 ĐÈN NGỦ SURIGHT hàng chính hãng,830000,high_>700k,0.0,0,https://tiki.vn/set-2-den-ngu-suright-hang-chi...,SURIGHT,Telovietnam,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,"Đèn ngủ LED phát ra ánh sáng trắng ấm, mềm mại..."


In [103]:
test_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 18 non-null     int64  
 1   name               18 non-null     object 
 2   price              18 non-null     int64  
 3   price_bucket       18 non-null     object 
 4   rating             18 non-null     float64
 5   review_count       18 non-null     int64  
 6   url                18 non-null     object 
 7   brand_detail       18 non-null     object 
 8   seller_detail      17 non-null     object 
 9   categories         17 non-null     object 
 10  category_root      0 non-null      float64
 11  category_leaf      17 non-null     object 
 12  short_description  18 non-null     object 
dtypes: float64(2), int64(3), object(8)
memory usage: 2.0+ KB


In [104]:
print("\n === BẮT ĐẦU CRAWL CHÍNH ===\n")

for group, kws in HOME_LIFE_KEYWORDS.items():
    for kw in kws:
        search_tiki_full(kw, group, max_pages=15, checkpoint_every=1000)
        _sleep(4, 7)


 === BẮT ĐẦU CRAWL CHÍNH ===



Dụng cụ nhà bếp → ấm nước các loại: 100%|██████████| 15/15 [14:05<00:00, 56.39s/it]


Hoàn thành keyword: ấm nước các loại (1 checkpoint)



Dụng cụ nhà bếp → dụng cụ chứa đựng thực ph: 100%|██████████| 15/15 [16:40<00:00, 66.68s/it]


Hoàn thành keyword: dụng cụ chứa đựng thực phẩm (1 checkpoint)



...

[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 1
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 2
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 1
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 2
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 3
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 4
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 5


Dụng cụ & Thiết bị tiện ích → miếng dán đa năng:  73%|███████▎  | 11/15 [28:15<09:12, 138.25s/it]

[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 1
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 2
[TIMEOUT] https://tiki.vn/api/v2/reviews | retry 3


Dụng cụ & Thiết bị tiện ích → miếng dán đa năng: 100%|██████████| 15/15 [37:35<00:00, 150.40s/it]


Hoàn thành keyword: miếng dán đa năng (1 checkpoint)



In [105]:
def merge_checkpoints():
    product_files = [f"./data_full_2/checkpoint/{f}" for f in os.listdir("./data_full_2/checkpoint") if f.startswith("products_ckpt_")]
    review_files  = [f"./data_full_2/checkpoint/{f}" for f in os.listdir("./data_full_2/checkpoint") if f.startswith("reviews_ckpt_")]

    if not product_files or not review_files:
        print("Không có file checkpoint để gộp.")
        return

    df_products = pd.concat([pd.read_csv(f) for f in product_files], ignore_index=True).drop_duplicates(subset=["id"])
    df_reviews  = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)

    df_products.to_csv("./data_full_2/final/home_life_products_details.csv", index=False, encoding="utf-8-sig")
    df_reviews.to_csv("./data_full_2/final/home_life_reviews.csv", index=False, encoding="utf-8-sig")

    print(f"\nGộp xong tất cả checkpoint:")
    print(f"{len(df_products)} sản phẩm, {len(df_reviews)} review")
    print("Lưu tại: ./data_full_2/final/home_life_products_details.csv & home_life_reviews.csv")

merge_checkpoints()

Không có file checkpoint để gộp.


In [106]:
def merge_checkpoints():
    product_files = [f"./data_full/checkpoint/{f}" for f in os.listdir("./data_full/checkpoint") if f.startswith("products_ckpt_")]
    review_files  = [f"./data_full/checkpoint/{f}" for f in os.listdir("./data_full/checkpoint") if f.startswith("reviews_ckpt_")]

    if not product_files or not review_files:
        print("Không có file checkpoint để gộp.")
        return

    df_products = pd.concat([pd.read_csv(f) for f in product_files], ignore_index=True).drop_duplicates(subset=["id"])
    df_reviews  = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)

    df_products.to_csv("./data_full/final/home_life_products_details.csv", index=False, encoding="utf-8-sig")
    df_reviews.to_csv("./data_full/final/home_life_reviews.csv", index=False, encoding="utf-8-sig")

    print(f"\nGộp xong tất cả checkpoint:")
    print(f"{len(df_products)} sản phẩm, {len(df_reviews)} review")
    print("Lưu tại: ./data_full/final/home_life_products_details.csv & home_life_reviews.csv")

merge_checkpoints()


Gộp xong tất cả checkpoint:
11746 sản phẩm, 53009 review
Lưu tại: ./data_full/final/home_life_products_details.csv & home_life_reviews.csv


In [107]:
df_product=pd.read_csv("./data_full/final/home_life_products_details.csv")

In [108]:
df_product

,id,name,price,price_bucket,rating,review_count,brand,seller,url,kw,group,page,brand_detail,seller_detail,categories,category_root,category_leaf,short_description
0,101995256,Thanh Xốp Chắn Khe Cửa Đa Năng Thông Minh,22400,low_<100k,4.7,39,PEAFLO,NaN,https://tiki.vn/thanh-xop-chan-khe-cua-da-nang...,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích,1,PEAFLO,Gold,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,Thanh Xốp Chắn Khe Cửa Đa Năng Thông Minh Cao ...
1,276555266,COMBO 2 Thanh Nẹp Chặn Khe Cửa - Ron Đệm Cửa C...,78400,low_<100k,0.0,0,LuckyJQR,NaN,https://tiki.vn/combo-2-thanh-nep-xop-chan-khe...,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích,1,LuckyJQR,OM LUCKY WAHIK,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,Đặc điểm nổi bật: do sản phẩm được làm bằng xố...
2,59598451,"Miếng xốp bọc chân cửa chắn gió chống bụi, ngă...",12703,low_<100k,4.8,12,OEM,NaN,https://tiki.vn/mieng-xop-boc-chan-cua-chan-gi...,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích,1,OEM,Ghế Nam Việt,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,"Miếng xốp bọc chân cửa chắn gió chống bụi, ngă..."
3,278431819,BỘ BÁO ĐỘNG CHỐNG TRỘM GẮN CỬA NHÀ CỬA SỔ BẢO ...,79200,low_<100k,0.0,0,OEM,NaN,https://tiki.vn/ban-p278431819.html?spid=27843...,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích,1,OEM,SHOP MẠNH TÂM,Bàn ghế làm việc,NaN,Bàn ghế làm việc,chống trộmSiêu đẹp siêu chất siêu bền siêu rẻ ...
4,262304471,Miếng Chặn Cửa Cao Su Chống Va Đập KUNBE Loại ...,9600,low_<100k,4.0,10,KUNBE,NaN,https://tiki.vn/mieng-chan-cua-cao-su-chong-va...,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích,1,KUNBE,chat24h,Bảo vệ nhà cửa khác,NaN,Bảo vệ nhà cửa khác,Miếng Chặn Cửa Cao Su Chống Va Đập KUNBE Loại ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11741,160562074,Nhang nụ siêu sạch gỗ Ngọc Am lên tuyết - hươn...,20800,low_<100k,4.0,1,HAHANCO,NaN,https://tiki.vn/nhang-nu-sieu-sach-go-ngoc-am-...,"ống cắm hương, nhang",Đồ thờ cúng,13,HAHANCO,Đức Tuấn Shop,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,NHANG NỤ SẠCH GỖ NGỌC AM NGUYÊN CHẤT KHÓI TỎA ...
11742,275608026,Nhang trầm hương Sao Việt - Loại 3 cây 4 tấc -...,72000,low_<100k,5.0,1,TRẦM HƯƠNG SAO VIỆT,NaN,https://tiki.vn/nhang-tram-huong-sao-viet-loai...,"ống cắm hương, nhang",Đồ thờ cúng,14,TRẦM HƯƠNG SAO VIỆT,TRẦM HƯƠNG SAO VIỆT,"Hương, nhang",NaN,"Hương, nhang",Giới thiệu về nhang trầm 3 cây cỡ lớnNhang trầ...
11743,277620428,Nhang trầm ống KHÔNG TĂM cao cấp - 100% tự nhi...,54400,low_<100k,0.0,0,Tam Anh,NaN,https://tiki.vn/nhang-tram-ong-khong-tam-cao-c...,"ống cắm hương, nhang",Đồ thờ cúng,14,Tam Anh,Vòng Gỗ Tâm Anh,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,Nhang trầm Tâm Anh loại NHANG KHÔNG TĂM cao cấ...
11744,9666396,Nhang sạch thảo dược Kính Tâm hộp gỗ hương,250000,mid_100k_300k,0.0,0,Hương Xưa,NaN,https://tiki.vn/nhang-sach-thao-duoc-kinh-tam-...,"ống cắm hương, nhang",Đồ thờ cúng,14,Hương Xưa,Hương Xưa,Nhà Cửa - Đời Sống,NaN,Nhà Cửa - Đời Sống,Hương nhang Thảo dược sạch tự nhiên - Hương tr...


In [111]:
df_review=pd.read_csv("./data_full/final/home_life_reviews.csv")

In [112]:
df_review

,product_id,customer_id,customer_name,rating,title,content,thank_count,created_at_raw,event_time,hour,day_of_week,is_weekend,time_slot,context_key,kw,group
0,101995256,30416766.0,\tDươNG THị YếN NGọC,5,Cực kì hài lòng,Rất tốt.,0,1751100239,2025-06-28T15:43:59,15,5,1,afternoon,afternoon_wd5,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích
1,101995256,8366209.0,Hồ Bảo Triệu Duy,5,Cực kì hài lòng,Hàng dùng đc giá theo chất lượng hàng,0,1741863537,2025-03-13T17:58:57,17,3,0,afternoon,afternoon_wd3,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích
2,101995256,29332709.0,Huỳnh Tiến Đạt,4,Hài lòng,NaN,0,1757716648,2025-09-13T05:37:28,5,5,1,morning,morning_wd5,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích
3,101995256,16704174.0,Lê Anh Vân Hương,4,Hài lòng,NaN,0,1757080962,2025-09-05T21:02:42,21,4,0,evening,evening_wd4,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích
4,101995256,28449953.0,Phan thị thu sương,5,Cực kì hài lòng,NaN,0,1755614063,2025-08-19T21:34:23,21,1,0,evening,evening_wd1,bảo vệ nhà cửa,Dụng cụ & Thiết bị tiện ích
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53004,137226640,20891998.0,Le Tâm,5,Cực kì hài lòng,"Tiki giao hàng nhanh, chưa xài nên chưa biết t...",0,1644298740,2022-02-08T12:39:00,12,1,0,noon,noon_wd1,"ống cắm hương, nhang",Đồ thờ cúng
53005,137226640,11103373.0,Hoang Ha,4,Hài lòng,NaN,0,1662565660,2022-09-07T22:47:40,22,2,0,late_night,late_night_wd2,"ống cắm hương, nhang",Đồ thờ cúng
53006,137226640,17764292.0,Đỗ Hoàng Phong,4,Hài lòng,NaN,0,1662068915,2022-09-02T04:48:35,4,4,0,late_night,late_night_wd4,"ống cắm hương, nhang",Đồ thờ cúng
53007,192850357,8917895.0,Vu Thi Truc Oanh,5,Cực kì hài lòng,"Hỏi thăm shop trả lời rất nhanh, gửi hàng cũng...",0,1690037564,2023-07-22T21:52:44,21,5,1,evening,evening_wd5,"ống cắm hương, nhang",Đồ thờ cúng


In [113]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11746 entries, 0 to 11745
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 11746 non-null  int64  
 1   name               11746 non-null  object 
 2   price              11746 non-null  int64  
 3   price_bucket       11746 non-null  object 
 4   rating             11746 non-null  float64
 5   review_count       11746 non-null  int64  
 6   brand              11231 non-null  object 
 7   seller             0 non-null      float64
 8   url                11746 non-null  object 
 9   kw                 11746 non-null  object 
 10  group              11746 non-null  object 
 11  page               11746 non-null  int64  
 12  brand_detail       11230 non-null  object 
 13  seller_detail      11591 non-null  object 
 14  categories         11591 non-null  object 
 15  category_root      0 non-null      float64
 16  category_leaf      115

In [114]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53009 entries, 0 to 53008
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      53009 non-null  int64  
 1   customer_id     52950 non-null  float64
 2   customer_name   52959 non-null  object 
 3   rating          53009 non-null  int64  
 4   title           53007 non-null  object 
 5   content         28887 non-null  object 
 6   thank_count     53009 non-null  int64  
 7   created_at_raw  53009 non-null  int64  
 8   event_time      53009 non-null  object 
 9   hour            53009 non-null  int64  
 10  day_of_week     53009 non-null  int64  
 11  is_weekend      53009 non-null  int64  
 12  time_slot       53009 non-null  object 
 13  context_key     53009 non-null  object 
 14  kw              53009 non-null  object 
 15  group           53009 non-null  object 
dtypes: float64(1), int64(7), object(8)
memory usage: 6.5+ MB
